## PROTOCOL
<span class="mark">왜 python에서는 object.length() 대신에 len(object)를 사용하나요?</span>

- 특정한 action에 대해 python 언어가 제시하는 protocol이 있습니다.
 - <span class="girk">for문에서 iterable한 객체를 다루는 protocol을 떠올려보세요.</span>
- 해당 protocol을 구현만 한다면 어느 객체도 python이 다룰 수 있습니다.
 - 사용자 객체도 builtin객체처럼 다룰 수 있는 강력한 장점을 제시합니다.
 - builtin처럼 동작시키기 위해 기존 상속 메커니즘(혹은 java에서 사용하는 interface)을 필요로 하지 않습니다.
 - 그저 *필요한 동작에 대한 protocol만 구현해주면 됩니다.*
- dunder function 스타일로 구현합니다.

In [1]:
# 예) 객체의 길이계산은 __len__을 호출합니다.
class Sample():
    def __len__(self):
        return 10

len(Sample())    

10

In [5]:
# 예) 호출은 __call__을 호출합니다.
# 객체 인스턴스를 함수처럼 사용하기
class Sample():
    def __call__(self):
        return 10
    
sample = Sample()
sample()

10

In [8]:
# 예) in operator는 __contains__를 호출합니다.
class Sample():
    def __contains__(self, any_value):
        return True
sample = Sample()
10 in sample, 20 in sample, 30 in sample

(True, True, True)

In [11]:
# 이상한 객체를 만들 수 있습니다.
#__setattr__, __getattr__을 배우기 위함.
class Blackhole():
    def __init__(self):
        self.hello = 'world'
    def __getattr__(self, name):
        return 'Universe'
    
b = Blackhole()
b.hello = 'world'
b.hello


RecursionError: maximum recursion depth exceeded

In [ ]:
# 이상한 객체를 만들 수 있습니다.
#__getitem__, __setitem__을 배우기 위함.
class AnotherBlackhole():
    def __getitem__(self, name):
        return 'Universe'

b = AnotherBlackhole()
# [] operator로 접근할 때.
b['hello'], b['world']

특정 protocol 구현여부를 검사할 수도 있습니다. (특정한 동작을 수행할 수 있는지)

In [8]:
from collections.abc import Sized, Callable
from collections import abc
print(dir(abc))

# 예) 객체의 길이계산은 __len__을 호출합니다.
class Sample:
    def __len__(self):
        return 10
    def __call__(self):
        return 20

sample = Sample()
isinstance(sample, Sized) and len(sample), \
isinstance(sample, Callable) and sample()

['AsyncGenerator', 'AsyncIterable', 'AsyncIterator', 'Awaitable', 'ByteString', 'Callable', 'Collection', 'Container', 'Coroutine', 'Generator', 'Hashable', 'ItemsView', 'Iterable', 'Iterator', 'KeysView', 'Mapping', 'MappingView', 'MutableMapping', 'MutableSequence', 'MutableSet', 'Reversible', 'Sequence', 'Set', 'Sized', 'ValuesView', '__all__', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__']


(10, 20)

In [9]:
from collections.abc import Container
class Sample:
    def __contains__(self, _):
        return True

isinstance(Sample(), Container)

True

만약 어떤 새가 오리처럼 걷고, 헤엄치고, 꽥꽥거리는 소리를 낸다면 나는 그 새를 오리라고 부를 것이다.

- duck typing

> 컴퓨터 프로그래밍 분야에서 덕 타이핑(duck typing)은 동적 타이핑의 한 종류로, 객체의 변수 및 메소드의 집합이 객체의 타입을 결정하는 것을 말한다. <u>클래스 상속이나 인터페이스 구현으로 타입을 구분하는 대신, 덕 타이핑은 객체가 어떤 타입에 걸맞은 변수와 메소드를 지니면 객체를 해당 타입에 속하는 것으로 간주한다.</u> “덕 타이핑”이라는 용어는 다음과 같이 표현될 수 있는 덕 테스트에서 유래했다. (덕은 영어로 오리를 의미한다.)

 - type을 이용한 비교를 잘 사용하지 않게 됩니다. 상속을 고려해주지 않기 때문입니다.
 - python모델은 가능한 많이 핵심 프로토콜과 협업하려고 애씁니다. 최소한의 인터페이스만 구현되어 있으면 그것을 최대한으로 활용해서 모델로서 동작하도록 구현되어있습니다.
 - python 클래스는 동적 변경이 가능하기 때문에 (monkey patching) duck-typing을 유연하게 사용할 수 있습니다. 어떤 객체든 인터페이스를 갖고 있으면 동작할 것이고, 그것이 최초에 가지고 있었던 후에 동적으로 추가되었던 상관하지 않기 때문입니다.

그래도 굳이 내가 만든 함수에서 arguement의 type검사를 해본다고 합시다.

In [1]:
# 나는 dict 형식의 argument만 받아서 처리하고 싶은 함수
# 이렇게 잘 하지 않습니다만,
def proc_dict(d):  # OrderedDict, Dict
     if type(d) == type({}):
            pass

In [7]:
# builtin-dictionary와 호환되는 object를 만들고 싶다면
from collections.abc import Mapping
from collections import OrderedDict

# 다음 세개 method중 하나라도 없으면 에러발생!
class MyDict(Mapping):
    def __getitem__(self, key):
        return None
    def __iter__(self):
        pass
    def __len__(self):
        return 0

print(isinstance({'hello': 'world'}, Mapping))
print(isinstance(OrderedDict({'hello': 'world'}), Mapping))
print(isinstance(MyDict(), Mapping))

True
True
True


In [2]:
# https://docs.python.org/3/library/collections.abc.html
from collections.abc import Iterator, Iterable

class MyIterableObject(Iterator):
    pass

MyIterableObject()

TypeError: Can't instantiate abstract class MyIterableObject with abstract methods __next__

In [3]:
# https://docs.python.org/3/library/collections.abc.html
from collections.abc import Sized

class MySizeCheckableObject(Sized):
    pass

MySizeCheckableObject()

TypeError: Can't instantiate abstract class MySizeCheckableObject with abstract methods __len__

Decorator again! 클래스로도 구현할 수 있습니다.

In [2]:
import functools
def decorator(func):
    @functools.wraps(func)
    def inner(*args, **kwargs):
        print('before function call')
        func(*args, **kwargs)
        print('after function call')
    return inner

In [1]:
class decorator_class(object):
    def __init__(self, function):
        self.function = function
    def __call__(self, *args, ** kwargs):
        # do something before invocation 
        result = self.function(*args, ** kwargs)
        # do something after
        return result

## EXERCISE

In [4]:
# pypi를 관리하는 class를 만들어봅시다.

# url은 다음 형태로 관리됩니다.
# https://pypi.python.org/pypi/Flask/0.12.2/json



In [5]:
import json
from urllib.request import urlopen
from urllib.error import HTTPError

class PyPiError(Exception):
    pass

class PyPi:
    def __init__(self, name, version=None):
        base_url = 'https://pypi.python.org/pypi/'
        if version:
            url = base_url + name + '/' + version + '/json'
        else:
            url = base_url + name + '/json'
        try:
            self._data = json.load(urlopen(url))
        except HTTPError:
            raise PyPiError(f'{name} does not exists')
        self.name = name
        self.version = self._data['info']['version']

    def to_json(self):
        return json.dumps(self._data)
    
    def __repr__(self):
        return f"PyPi({self.name}, {self.version})"
    
    def __str__(self):
        return f"PyPi({self.name}, {self.version}):str"

    def __getattr__(self, key):
        return self._data['info'][key]
    
    def __len__(self):
        return len(self._data)
    
    def __iter__(self):
        return iter(self._data['info'])

    def __contains__(self, key):
        return key in self._data
    
########################################################################
# 다음 테스트를 통과하도록 작성해보세요.
pypi = PyPi('Flask')

# 다음 세가지 속성을 사용할 수 있습니다. @property
assert(pypi.author == 'Armin Ronacher')
assert pypi.home_page == "http://github.com/pallets/flask/"
assert pypi.version == '0.12.2'

# # 다음 메서드를 만들어주세요.
assert pypi.to_json()

# 버전을 추가로 지정할 수 있습니다.
pypi = PyPi('Flask', '0.9')

assert pypi.version == '0.9'

# # 존재하지 않는 버전이나 패키지를 찾을시에 오류발생
# pypi = PyPi('Flaask')
# raise Exception("Flaask doesn't exist.")
# pypi = PyPi('Flask', '99.99')
# raise Exception("Flask(99.99) doesn't exist.")

# # 문자열표현을 만들어보아요.
str(pypi), pypi
# # > "PyPi(Flask, 0.12.2)"

# # info의 모든 속성을 객체의 속성으로 연결해보세요.
pypi.author_email, pypi.platform
# # ...

# # json데이터의 length를 계산해보세요.
len(pypi)

# # for-loop을 통해 info내용을 볼 수 있도록 만들어보아요.
for key in pypi._data['info']:
# for key in pypi:
    print(key)
    
# # 특정 key값을 검사할 수 있는 기능을 만들어보아요.
'x' in pypi

AssertionError: 